In [1]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import yaml
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import itertools
import json
import random
import math
from google.colab import files
from google.colab import drive
import gspread
from google.auth import default
import ast
from itertools import product
from google.colab import drive
from pathlib import Path


In [2]:
!git clone https://github.com/felixludos/omni-fig
!pip install -e ./omni-fig

Cloning into 'omni-fig'...
remote: Enumerating objects: 2330, done.
remote: Counting objects: 100% (105/105), done.
remote: Compressing objects: 100% (77/77), done.
remote: Total 2330 (delta 50), reused 69 (delta 24), pack-reused 2225
Receiving objects: 100% (2330/2330), 2.81 MiB | 12.71 MiB/s, done.
Resolving deltas: 100% (1493/1493), done.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Obtaining file:///content/omni-fig
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.3/63.3 kB 823.8 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 6.6 MB/s eta 0:00:00
  Created wheel for omnibelt: filename=omnibelt-0.7.4-py3-none-any.whl size=73962 sha256=9f9b57e3a6a7f3c8bbccb829118d048a2690cc488f19c19c6635ad54c3fbd072
  Stored in directory: /root/.cache/pip/wheels/03/52/64/362a96515ce087df37913bdcdd4bd2334e42016cb1c496dd36
Successfully buil

In [3]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

creds, _ = default()
gc = gspread.authorize(creds)

In [7]:
os.getcwd()

'/content'

In [8]:
#defining my worksheet
graph2story = gc.open('20230212_Elements_for_Data_Generation').worksheet('graph2story_finer')
graph2temp = gc.open('20230212_Elements_for_Data_Generation').worksheet('graph2temp_finer')
graph2query = gc.open('20230212_Elements_for_Data_Generation').worksheet('query_dependencies')
query2cheatsheet = gc.open('20230212_Elements_for_Data_Generation').worksheet('cheatsheets')

In [9]:
graph2story_data = graph2story.get_all_values()
graph2temp_data = graph2temp.get_all_values()
graph2query_data = graph2query.get_all_values()
query2cheatsheet_data = query2cheatsheet.get_all_values()
graph2query_dict = [dict(zip(graph2query_data[1], row)) for row in graph2query_data[2:]]
query2cheatsheet_dict = [dict(zip(query2cheatsheet_data[0], row)) for row in query2cheatsheet_data[1:]]
graph2story_dict = [dict(zip(graph2story_data[1], row)) for row in graph2story_data[2:]]

In [10]:
# query2cheatsheet_dict[0]

In [11]:
for dct in graph2query_dict:
    phenomenon = [p.strip() for p in dct['phenomenon'].split(',')]
    dct['phenomenon'] = phenomenon

In [12]:
for i in range(len(graph2query_dict)):
  if len(graph2query_dict[i]['query_id'])> 0:
    data = {'query_id': graph2query_dict[i]['query_id'],
            'query_formal_form': graph2query_dict[i]['query_formal_form'],
            'query_word_form': graph2query_dict[i]['query_word_form'],
            'phenomenon': graph2query_dict[i]['phenomenon'],
            'CI_engine': graph2query_dict[i]['CI_engine_to_get_the_answer'],
            'cheatsheet': [c for c in query2cheatsheet_dict if c['query_id'] == graph2query_dict[i]['query_id']]
    }

  with open(f"/content/drive/MyDrive/ymls/questions/{graph2query_dict[i]['query_id']}.yml",'w') as f:
    yaml.dump(data,f)
  f.close()





In [13]:
for dct in graph2story_dict:
    queries = [q.strip() for q in dct['queries'].split(',')]
    dct['queries'] = queries

In [16]:
graph2story_dict_expand = graph2story_dict.copy()

for row in graph2story_dict:
  if row['params']:
    params_dict = ast.literal_eval(row['params'])
    # Generate all possible combinations of values in the 'params' dictionary
    param_combinations = list(product(*params_dict.values()))

    # Create a list of new dictionaries with the different combinations of 'params'
    new_dicts = []
    for param_values in param_combinations:
      new_dict = row.copy()
      new_dict['params'] = dict(zip(eval(new_dict['params']).keys(), param_values))
      if 'negation' in new_dict['params'] and new_dict['params']['negation'] == True:
        new_dict['story_id'] = f"{new_dict['story_id']}_not"
      if 'conjunction' in new_dict['params'] and new_dict['params']['conjunction'] == True:
        new_dict['story_id'] = f"{new_dict['story_id']}_and"
      elif 'conjunction' in new_dict['params'] and new_dict['params']['conjunction'] == False:
        new_dict['story_id'] = f"{new_dict['story_id']}_or"

      new_params = dict(zip(params_dict.keys(), param_values))
      new_dict['params'] = new_params
      graph2story_dict_expand.append(new_dict)

filtered_expand = [d for d in graph2story_dict_expand if '[' not in d['params'] ]
    
# Print the list of new dictionaries

print(len(filtered_expand))
print(len(graph2story_dict))
print(len(graph2story_dict_expand))


78
36
88


In [17]:
graph2story_dict = filtered_expand.copy()

In [18]:
story_ids = [d['story_id'] for d in graph2story_dict]
len(set(story_ids))

52

In [19]:
print(len(graph2story_dict))

78


In [20]:
prop_list = []
for i in range(len(graph2story_dict)):
  data = {
      'meta':{
        'phenomenon': graph2story_dict[i]['phenomenon']
            },
      'story_id': graph2story_dict[i]['story_id'],
      'params':graph2story_dict[i]['params'],
      'queries': graph2story_dict[i]['queries'],
      'scm': graph2story_dict[i]['phenomenon'],
      'labels':{
        'Xname': graph2story_dict[i]['Xname'],
        'X0':  graph2story_dict[i]['X0'],
        'X1': graph2story_dict[i]['X1'],
        'Zname': graph2story_dict[i]['Zname'],
        'Z0': graph2story_dict[i]['Z0'],
        'Z1': graph2story_dict[i]['Z1'],
        'Yname': graph2story_dict[i]['Yname'],
        'Y0': graph2story_dict[i]['Y0'],
        'Y1': graph2story_dict[i]['Y1'],
        'Wname': graph2story_dict[i]['Wname'],
        'W0': graph2story_dict[i]['W0'],
        'W1': graph2story_dict[i]['W1'],
      },
      'variable_correspondence': graph2story_dict[i]['variable_correspondence'],
      'Source': graph2story_dict[i]['Source']
          }
  prop_list.append(row['story_id'])

  with open(f"/content/drive/MyDrive/ymls/stories/{graph2story_dict[i]['story_id']}.yml",'w') as f:
    yaml.dump(data,f)
  f.close()

       

In [21]:
len(prop_list)

78